# Predicting House Price using NeuralNet Regressor
There are 5 parts in this notebook.

0. Preprocess tne dataset
1. Training on plaintext data
2. Evaluate model on plaintext data
3. Train and Quantize the Concrete model (Quantization Aware Training)
4. Compile the model to the equivalent FHE circuit
5. Evaluate the FHE model on encrypted data
6. MAE measurement

In [45]:
# import required packages
import time
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import NeuralNetRegressor as ConcreteNNRegressor
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [46]:
# import dataset as panda's dataframe
import pandas as pd
house_dataset = pd.read_csv("./data/house_dataset.csv")
df = pd.DataFrame(house_dataset)
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2915,160,RM,21,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2916,2916,160,RM,21,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2917,2917,20,RL,160,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2918,2918,85,RL,62,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


## 0. Preprocess the dataset

In [51]:
# Separate features and target
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# Automatically find categorical columns (assuming they're of type object)
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Define ColumnTransformer
# This transformer applies OneHotEncoder to categorical features and 'passthrough' for all other columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)


# Create a pipeline with preprocessing and a model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(max_iter=1000))
])
# mean = target.mean()
# std = target.std()
# house_dataset = house_dataset[target < mean + 3*std]
# target = target[target < mean + 3*std]

# # print(house_dataset)
# plt.hist(target, bins=1000,edgecolor='black')
# plt.show()

X


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2915,160,RM,21,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,2916,160,RM,21,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,2917,20,RL,160,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
2918,2918,85,RL,62,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [49]:
# split dataset into 80% training data and 20% testing data
# house_dataset = np.array(house_dataset)
# target = np.array(round(target*100))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Fit the pipeline
pipeline.fit(X_train, y_train)

ValueError: Input y contains NaN.

## 1. Training on plaintext data

In [18]:
# train sklearn MLPRegressor model on the clear
sklearn_MLP_regressor = SklearnMLPRegressor(
                    alpha=1,
                    activation="identity",
                    max_iter=1000,
                    hidden_layer_sizes=(25,),
                    learning_rate_init=0.005,)
sklearn_MLP_regressor.fit(X_train, y_train)

ValueError: could not convert string to float: 'FV'

## 2. Evaluate model on plaintext data

In [14]:
time_begin = time.time()
y_pred = sklearn_MLP_regressor.predict(X_test)
print(y_pred)
execution_time_on_plaintext = (time.time() - time_begin) / len(X_test)
print(f"Model evaluation time on the clear: {execution_time_on_plaintext:.8f} seconds per sample")
# Compute the R2 scores
sklearn_r2_score = r2_score(y_test, y_pred)
print(sklearn_r2_score)

[11.29941699 11.29950719 36.79874836 ... 11.61936247 24.79950316
 12.95937163]
Model evaluation time on the clear: 0.00000024 seconds per sample
0.999999994683011


## 3. Train and Quantize the Concrete model (Quantization Aware Training)

In [15]:
# Instantiate the model with parameter
# TODO: tune the parameter
params_neural_net = {
    # "module__n_w_bits": 6,
    # "module__n_a_bits": 8,
    # "module__n_accum_bits": 16,
    "module__n_hidden_neurons_multiplier": 10,
    "module__n_layers": 2,  # total number of layers in the FCNN = 1 hidden layer
    "module__activation_function": torch.nn.ReLU,
    "max_epochs": 1,
    "verbose": 1,
    # "lr": 0.1,
}


#some sort of Feature preprocessing needed for quantization aware training
# Linear models require polynomial features to be applied before training to fit a non-linear model and other models perform better with this transoformation
pipe = Pipeline(
    [
        ("poly", PolynomialFeatures()),
        ("scaler", StandardScaler()),
    ]
)

X_poly_train = pipe.fit_transform(X_train)
X_poly_test = pipe.transform(X_test)

concrete_NN_regressor = ConcreteNNRegressor(batch_size=32, **params_neural_net)

# train the concrete linear regression model on clear data
# The built-in NN regressor models will automatically quantize weights and activations with .fit() call. (Quantization Aware Training) These models use several layers for Quantization Aware Training, allowing good performance for low precision (down to 2-3 bits) weights and activations.
# The maximum accumulator bit-width is controlled by the number of weights and activation bits, as well as a pruning factor. This factor is automatically determined based on the desired accumulator bit-width and a multiplier factor can be optionally specified.


concrete_NN_regressor.fit(X_poly_train, y_train.values.reshape(-1, 1))
print("done training")

# Now, we can test our Concrete ML model on the clear test data
y_pred_q = concrete_NN_regressor.predict(X_poly_test)
print("done prediction")
# Compute the R2 scores
quantized_r2_score = r2_score(y_test, y_pred_q)

print("done calculating r2 score")

print(quantized_r2_score)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1       11.7663       40.5754  57.8481
done training
done prediction
done calculating r2 score
0.6659336746804537


## 4. Compile the model to the equivalent FHE circuit

In [16]:
# Compile the quantized model in to FHE circuit and run inference on it
# You have to provide the training dataset in order to compile the quantized model to equivalent FHE circuit
time_begin = time.time()
fhe_circuit = concrete_NN_regressor.compile(X_poly_train)
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")
print(f"Compilation time: {time.time() - time_begin:.4f} seconds")

# Compiler returns the circuit, which can be used to generated a secrete key and evaluation key
# secrete key: used for encryption and decryption. only accesible to the client
# evaluation key: used to evaluate the cirucit on encypted data. anyone can access it
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")

time_begin = time.time()
fhe_circuit.client.keygen(force=False)
print(f"Key generation time: {time.time() - time_begin:.4f} seconds")

Generating a key for a 12-bit circuit
Compilation time: 248.9701 seconds
Generating a key for a 12-bit circuit
Key generation time: 154.4244 seconds


## 5. Evaluate the FHE model on encrypted data

In [17]:
# Evaluate the FHE-version of the model
time_begin = time.time()
y_pred_fhe = concrete_NN_regressor.predict(X_poly_test[:1], fhe="execute")

execution_time_on_ciphertext = (time.time() - time_begin) / len(X_test)
print(f"Execution time: {execution_time_on_ciphertext:.8f} seconds per sample")
print(f"which is {(execution_time_on_ciphertext / execution_time_on_plaintext):.2f} times slower than prediction on the plaintext data")

: 

In [ ]:
# Measure R2 score of FHE version of the model
fhe_r2_score = r2_score(y_test, y_pred_fhe)

print("R^2 scores:")
print(f"scikit-learn (clear): {sklearn_r2_score:.4f}")
print(f"Concrete ML (quantized model on plaintext): {quantized_r2_score:.4f}")
print(f"Concrete ML (FHE model on ciphertext): {fhe_r2_score:.4f}")

# Measure the error of the FHE quantized model with respect to the clear scikit-learn float model
concrete_score_difference = abs(fhe_r2_score - quantized_r2_score) * 100 / quantized_r2_score
print(
    "\nRelative score difference for Concrete ML (quantized model on clear) vs. Concrete ML (FHE):",
    f"{concrete_score_difference:.2f}%",
)

# Measure the error of the FHE quantized model with respect to the clear float model
score_difference = abs(fhe_r2_score - sklearn_r2_score) * 100 / sklearn_r2_score
print(
    "Relative score difference for scikit-learn (clear) vs. Concrete ML (FHE) scores:",
    f"{score_difference:.2f}%",
)